In [1]:
import numpy as np

In [2]:
theta = np.array([1.7, 2, 7, 8])
rho = np.array([3.2, 14, 9, 2])
stepsize = 0.2

# metric = np.array([1, 1, 1, 1])
# metric = np.diag(metric)
metric = np.array([0.3, 2, 0.5, 0.1])
# metric = np.diag(metric)

In [3]:
def pi(theta):
    return np.exp(-0.5 * np.dot(theta, theta))

def grad(theta):
    return -theta


In [4]:
metric * grad(theta)

array([-0.51, -4.  , -3.5 , -0.8 ])

In [5]:
def bayeskit(theta, rho, grad, stepsize, metric):
    theta, rho = theta.copy(), rho.copy()
    
    rho -= 0.5 * stepsize * grad(theta)
    theta += stepsize * metric * rho
    print("Update: ", stepsize * metric * rho)
    rho -= 0.5 * stepsize * grad(theta)
    return theta, rho

In [6]:
def stan(theta, rho, grad, stepsize, metric):
    theta, rho = theta.copy(), rho.copy()
    
    rho -= 0.5 * stepsize * metric * grad(theta)
    print("Update: ", 0.5 * stepsize * metric * grad(theta))
    theta += stepsize * rho
    rho -= 0.5 * stepsize * metric @ grad(theta)
    return theta, rho

In [106]:
theta_bk, rho_bk = bayeskit(theta, rho, grad, stepsize, metric)
theta_stan, rho_stan = stan(theta, rho, grad, stepsize, metric)

print("Original:", theta, rho)
print("BayesKit:", theta_bk, rho_bk)
print("Stan:", theta_stan, rho_stan)

Update:  [0.2022 5.68   0.97   0.056 ]
Update:  [-0.051 -0.4   -0.35  -0.08 ]
Original: [1.7 2.  7.  8. ] [ 3.2 14.   9.   2. ]
BayesKit: [1.9022 7.68   7.97   8.056 ] [ 3.56022 14.968   10.497    3.6056 ]
Stan: [2.3502 4.88   8.87   8.416 ] [ 4.825166 15.974166 10.924166  3.654166]


In [107]:
theta_bk, rho_bk = bayeskit(theta, rho, grad, stepsize, np.diag(metric))
theta_stan, rho_stan = stan(theta, rho, grad, stepsize, np.diag(metric))

print("Original:", theta, rho)
print("BayesKit:", theta_bk, rho_bk)
print("Stan:", theta_stan, rho_stan)

ValueError: non-broadcastable output operand with shape (4,) doesn't match the broadcast shape (4,4)

# Sampling Dimensionality

In [5]:
seed = 1234
rng = np.random.default_rng(seed)
damping = 0.08
dim=4

rho_refresh = rng.normal(
    loc=rho * np.sqrt(1 - damping),
    scale=np.sqrt(damping * metric),
    size=dim
)

print(f"mean: {rho * np.sqrt(1 - damping)}")
print(f"std: {np.sqrt(damping * metric)}")
print(rho_refresh)

mean: [ 3.06933217 13.42832827  8.63249674  1.91833261]
std: [0.15491933 0.4        0.2        0.08944272]
[ 2.82086685 13.45396823  8.780675    1.93198328]


In [8]:
seed = 1234
rng = np.random.default_rng(seed)
damping = 0.08
dim=4

rho_refresh = rng.multivariate_normal(
    mean=rho * np.sqrt(1 - damping),
    cov=damping * np.diag(metric),
)

print(f"mean: {rho * np.sqrt(1 - damping)}")
print(f"std: {np.sqrt(damping * np.diag(metric))}")
print(rho_refresh)

mean: [ 3.06933217 13.42832827  8.63249674  1.91833261]
std: [[0.15491933 0.         0.         0.        ]
 [0.         0.4        0.         0.        ]
 [0.         0.         0.2        0.        ]
 [0.         0.         0.         0.08944272]]
[ 3.18411056 12.78679354  8.64531672  1.93198328]
